## Homework: Deep Jazz

In [1]:
import numpy as np
from music21 import stream, midi, tempo, note

from grammar import unparse_grammar
from preprocess import get_musical_data, get_corpus_data

from qa import prune_grammar, prune_notes, clean_up_notes
from generator import __sample, __generate_grammar, __predict

In [2]:
N_epochs = 128  # default
data_fn = 'midi/' + 'original_metheny.mid'  # 'And Then I Knew' by Pat Metheny
out_fn = 'midi/' 'deepjazz_on_metheny...' + str(N_epochs)

In [3]:
max_len = 20
max_tries = 1000
diversity = 0.5

# musical settings
bpm = 130

# get data
chords, abstract_grammars = get_musical_data(data_fn)
corpus, values, val_indices, indices_val = get_corpus_data(abstract_grammars)
print('corpus length:', len(corpus))
print('total # of values:', len(values))

corpus length: 193
total # of values: 78


In [4]:
chords[0]

[<music21.instrument.Piano Piano>,
 <music21.tempo.MetronomeMark Quarter=112.0>,
 <music21.key.Key of G major>,
 <music21.meter.TimeSignature 4/4>]

In [6]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM

def get_keras_model(max_len, N_values):
    # build a 2 stacked LSTM
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape=(max_len, N_values)))
    model.add(Dropout(0.2))
    model.add(LSTM(128, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(N_values))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    return model

Using TensorFlow backend.


## Task

Replace previous model with equivalent in prettytensor or tf.slim

Try to make you code as compact as possible

In [7]:
class DeepJazz:
    def __init__(max_len, N_values):
        self.max_len = max_len
        self.N_values = N_values
        self.dropout = tf.constant(0.2)
        
        self.x = tf.placeholder(tf.float32, shape=[None, max_len, N_values], name='x')
        self.y = tf.placeholder(tf.float32, [None, self.N_values], name = 'y')

        self.build()
        self.sess = tf.Session()
    
    def build():
        reset_graph()
        
        cell1 = tf.contrib.rnn.BasicLSTMCell(128)
        cell2 = tf.contrib.rnn.BasicLSTMCell(128)

        cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])
        cell = tf.nn.rnn_cell.DropoutWrapper(cell, input_keep_prob=self.dropout)
            
        init_state = cell.zero_state(batch_size, tf.float32)
        rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, self.x)
      
        logits = tf.contrib.layers.fully_connected(run_outputs, N_values, biases_initializer=tf.constant_initializer(0.0))
        self.predictions = tf.nn.softmax(logits)

        self.loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits, self.y))
    
    def create_batches(X, y, batch_size, epochs):
        indices = np.array(range(X.shape[0]))
        for i in range(epochs):
            cur_indeces = np.random_choice(indices, batch_size, replace=False)
            yield X[cur_indeces], y[cur_indeces]
    
    def fit(X, y, batch_size, epochs):
        global_step = tf.Variable(initial_value=0, name='global_step', trainable=False)
        optimizer = tf.train.RMSPropOptimizer(learning_rate=1e-4).minimize(self.loss, global_step=global_step)
        
        self.sess.run(tf.global_variables_initializer())
        for x_batch, y_batch in self.create_batches(X, y, batch_size, epochs):
            dict_ = {self.x: x_batch, self.y: y_batch}
            _, loss, num = self.sess.run([optimizer, self.loss, global_step]. dict_)
            if num % 1 == 0:
                print('num: {}; loss: {}'.format(num, loss))
        
    def predict(x, verbose):
        dict_ = {self.x: x}
        return sess.run([self.predictions], dict_)
        

In [8]:
# def get_pretty_tensor_model(max_len, N_values):
#     with tf.variable_scope('softmax'):


In [9]:
get_model = get_keras_model # get_pretty_tensor_model

In [10]:
import numpy as np

''' Build a 2-layer LSTM from a training corpus '''


def build_model(corpus, val_indices, max_len, N_epochs=128):
    # number of different values or words in corpus
    N_values = len(set(corpus))

    # cut the corpus into semi-redundant sequences of max_len values
    step = 3
    sentences = []
    next_values = []
    for i in range(0, len(corpus) - max_len, step):
        sentences.append(corpus[i: i + max_len])
        next_values.append(corpus[i + max_len])
    print('nb sequences:', len(sentences))

    # transform data into binary matrices
    X = np.zeros((len(sentences), max_len, N_values), dtype=np.bool)
    y = np.zeros((len(sentences), N_values), dtype=np.bool)
    for i, sentence in enumerate(sentences):
        for t, val in enumerate(sentence):
            X[i, t, val_indices[val]] = 1
        y[i, val_indices[next_values[i]]] = 1

    model = get_model(max_len, N_values)
    model.fit(X, y, batch_size=128, epochs=N_epochs)

    return model


In [ ]:
# build model
model = build_model(corpus=corpus, val_indices=val_indices,
                         max_len=max_len, N_epochs=N_epochs)

# set up audio stream
out_stream = stream.Stream()

# generation loop
curr_offset = 0.0
loopEnd = len(chords)
for loopIndex in range(1, loopEnd):
    # get chords from file
    curr_chords = stream.Voice()
    for j in chords[loopIndex]:
        curr_chords.insert((j.offset % 4), j)

    # generate grammar
    curr_grammar = __generate_grammar(model=model, corpus=corpus,
                                      abstract_grammars=abstract_grammars,
                                      values=values, val_indices=val_indices,
                                      indices_val=indices_val,
                                      max_len=max_len, max_tries=max_tries,
                                      diversity=diversity)

    curr_grammar = curr_grammar.replace(' A', ' C').replace(' X', ' C')

    # Pruning #1: smoothing measure
    curr_grammar = prune_grammar(curr_grammar)

    # Get notes from grammar and chords
    curr_notes = unparse_grammar(curr_grammar, curr_chords)

    # Pruning #2: removing repeated and too close together notes
    curr_notes = prune_notes(curr_notes)

    # quality assurance: clean up notes
    curr_notes = clean_up_notes(curr_notes)

    # print # of notes in curr_notes
    print('After pruning: %s notes' % (len([i for i in curr_notes
                                            if isinstance(i, note.Note)])))

    # insert into the output stream
    for m in curr_notes:
        out_stream.insert(curr_offset + m.offset, m)
    for mc in curr_chords:
        out_stream.insert(curr_offset + mc.offset, mc)

    curr_offset += 4.0

out_stream.insert(0.0, tempo.MetronomeMark(number=bpm))

# Play the final stream through output (see 'play' lambda function above)
play = lambda x: midi.realtime.StreamPlayer(x).play()
play(out_stream)

# save stream
mf = midi.translate.streamToMidiFile(out_stream)
mf.open(out_fn, 'wb')
mf.write()
mf.close()    

nb sequences: 58
Epoch 1/128
58/58 [==============================] - 2s 38ms/step - loss: 4.3555
Epoch 2/128
58/58 [==============================] - 0s 4ms/step - loss: 4.3201
Epoch 3/128
58/58 [==============================] - 0s 3ms/step - loss: 4.2725
Epoch 4/128
58/58 [==============================] - 0s 3ms/step - loss: 4.1163
Epoch 5/128
58/58 [==============================] - 0s 3ms/step - loss: 3.9251
Epoch 6/128
58/58 [==============================] - 0s 3ms/step - loss: 3.9870
Epoch 7/128
58/58 [==============================] - 0s 3ms/step - loss: 3.7089
Epoch 8/128
58/58 [==============================] - 0s 6ms/step - loss: 3.6216
Epoch 9/128
58/58 [==============================] - 0s 5ms/step - loss: 3.5484
Epoch 10/128
58/58 [==============================] - 0s 4ms/step - loss: 3.5581
Epoch 11/128
58/58 [==============================] - 0s 3ms/step - loss: 3.4660
Epoch 12/128
58/58 [==============================] - 0s 4ms/step - loss: 3.4219
Epoch 13/128
58/58 

58/58 [==============================] - 0s 3ms/step - loss: 0.5148
Epoch 103/128
58/58 [==============================] - 0s 2ms/step - loss: 0.4985
Epoch 104/128
58/58 [==============================] - 0s 2ms/step - loss: 0.5932
Epoch 105/128
58/58 [==============================] - 0s 3ms/step - loss: 0.5294
Epoch 106/128
58/58 [==============================] - 0s 2ms/step - loss: 0.6648
Epoch 107/128
58/58 [==============================] - 0s 2ms/step - loss: 0.6133
Epoch 108/128
58/58 [==============================] - 0s 3ms/step - loss: 0.5513
Epoch 109/128
58/58 [==============================] - 0s 2ms/step - loss: 0.4415
Epoch 110/128
58/58 [==============================] - 0s 3ms/step - loss: 0.3685
Epoch 111/128
58/58 [==============================] - 0s 3ms/step - loss: 0.3519
Epoch 112/128
58/58 [==============================] - 0s 3ms/step - loss: 0.3581
Epoch 113/128
58/58 [==============================] - 0s 3ms/step - loss: 0.3760
Epoch 114/128
58/58 [=========

You can play generated sample using any midi player

Under linux I prefer timidity

In [ ]:
!! timidity midi/deepjazz_on_metheny...128_epochs.midi